# Deep Learning
Date: September 20, 2023

In [23]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('font', size=18)
plt.rc('axes', titlesize=18)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)
plt.rc('legend', fontsize=18)
plt.rc('lines', markersize=10)

In [24]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

from tqdm import tqdm

In [25]:
mnist_trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
mnist_testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(mnist_trainset, batch_size=16, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(mnist_testset, batch_size=16, shuffle=False, num_workers=2)

In [26]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 100)
        self.fc2 = nn.Linear(100, 10)
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [27]:
device = torch.device("mps")
print("Device: ", device)

Device:  mps


In [28]:
writer = SummaryWriter()
net = Net()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [29]:
# update loss & accuracy each batch
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    progress_bar = tqdm(enumerate(trainloader), total=len(trainloader))
    for i, data in progress_bar:
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        description = f'Epoch: {epoch + 1}/{epochs} | Loss: {running_loss / (i + 1):.4f}'
        progress_bar.set_description(description)
        writer.add_scalar('training loss', loss.item(), epoch * len(trainloader) + i)
        writer.add_scalar('accuracy', (outputs.argmax(dim=1) == labels).float().mean().item(), epoch * len(trainloader) + i)
writer.flush()
writer.close()
print('Finished Training')

Epoch: 1/10 | Loss: 0.6450: 100%|██████████| 3750/3750 [00:33<00:00, 112.10it/s]
Epoch: 2/10 | Loss: 0.3071: 100%|██████████| 3750/3750 [00:35<00:00, 106.43it/s]
Epoch: 3/10 | Loss: 0.2568: 100%|██████████| 3750/3750 [00:33<00:00, 112.04it/s]
Epoch: 4/10 | Loss: 0.2230: 100%|██████████| 3750/3750 [00:33<00:00, 111.78it/s]
Epoch: 5/10 | Loss: 0.1982: 100%|██████████| 3750/3750 [00:31<00:00, 119.88it/s]
Epoch: 6/10 | Loss: 0.1794: 100%|██████████| 3750/3750 [00:32<00:00, 114.97it/s]
Epoch: 7/10 | Loss: 0.1632: 100%|██████████| 3750/3750 [00:30<00:00, 123.96it/s]
Epoch: 8/10 | Loss: 0.1497: 100%|██████████| 3750/3750 [00:30<00:00, 123.48it/s]
Epoch: 9/10 | Loss: 0.1384: 100%|██████████| 3750/3750 [00:30<00:00, 123.58it/s]
Epoch: 10/10 | Loss: 0.1285: 100%|██████████| 3750/3750 [00:32<00:00, 114.95it/s]

Finished Training
